In [1]:
'''

用戶菜單功能介紹

    用戶能透過點擊菜單，進行我方希冀之業務功能。
    
流程
    準備菜單的圖面設定檔
    讀取安全設定檔上的參數
    將菜單設定檔傳給Line
    對Line上傳菜單照片
    檢視現有的菜單
    將菜單與用戶做綁定
    將菜單與用戶解除綁定
    刪除菜單

'''

'\n\n用戶菜單功能介紹\n\n    用戶能透過點擊菜單，進行我方希冀之業務功能。\n    \n流程\n    準備菜單的圖面設定檔\n    讀取安全設定檔上的參數\n    將菜單設定檔傳給Line\n    對Line上傳菜單照片\n    檢視現有的菜單\n    將菜單與用戶做綁定\n    將菜單與用戶解除綁定\n    刪除菜單\n\n'

In [2]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "區塊鏈自定義菜單",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 5,
        "y": 0,
        "width": 824,
        "height": 850
      },
      "action": {
        "type": "message",
        "text": "[::text:]傳統交易"
      }
    },
    {
      "bounds": {
        "x": 0,
        "y": 850,
        "width": 825,
        "height": 818
      },
      "action": {
        "type": "message",
        "text": "[::text:]第三方公證人"
      }
    },
    {
      "bounds": {
        "x": 829,
        "y": 5,
        "width": 871,
        "height": 849
      },
      "action": {
        "type": "message",
        "text": "[::text:]多位公證人"
      }
    },
    {
      "bounds": {
        "x": 825,
        "y": 854,
        "width": 875,
        "height": 814
      },
      "action": {
        "type": "message",
        "text": "[::text:]多組織多位公證人"
      }
    },
    {
      "bounds": {
        "x": 1700,
        "y": 0,
        "width": 800,
        "height": 858
      },
      "action": {
        "type": "message",
        "text": "[::text:]教學訊息"
      }
    },
    {
      "bounds": {
        "x": 1700,
        "y": 858,
        "width": 800,
        "height": 810
      },
      "action": {
        "type": "message",
        "text": "[::text:]more"
      }
    }
  ]
}
"""

In [3]:
'''

讀取安全檔案內的字串，以供後續程式碼調用

'''
import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))


D9oCPKQF9qtKNNFE9Y7Xh90sm48FC/V02s8Q0kxa9c3WXQP/Zhr/7u0erHC1FiOKkNGRtPFuVcNSQgFeM6XR5QFvp2Z+CfAmYlLKw4zXtUCG/wzGedfxs3X8ltAdbsqigZRwBQlQwbFlV8BjVrf+KAdB04t89/1O/w1cDnyilFU=
1334661898c1758c8a899a5a4f603d48
U8633692756652c60054e28e65760bf8b


In [4]:
'''

將先前準備的菜單設定檔，以Post消息寄發給Line

    設定Line的遠端位置
    設定消息的基本安全憑證
    寄發消息，並取得回應之Id
    

'''

import requests

menuJson=json.loads(menuRawData)

createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

#print(createMenuRequestHeader)

lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)


<Response [200]>
{"richMenuId":"richmenu-63bc7a36bfe8fecc78e7915aa7307fa3"}


In [5]:
'''

將先前準備的菜單照片，以Post消息寄發給Line
    取得上面設定檔的Id
    設定Line的遠端位置
    設定消息的基本安全憑證
    上傳照片，並取得回傳成果

'''

# 取得菜單Id 
uploadRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")
print(uploadRichMenuId)
#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定Line的遠端位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 設定消息的基本安全憑證
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 上傳照片
uploadImageFile=open("../images/blockchain_demo.jpg",'rb')
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-63bc7a36bfe8fecc78e7915aa7307fa3
https://api.line.me/v2/bot/richmenu/richmenu-63bc7a36bfe8fecc78e7915aa7307fa3/content
<Response [200]>
{}


In [6]:
'''

將選單綁定到特定用戶身上
    取出上面得到的菜單Id及用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}

# 取得菜單id
linkRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")

# 將菜單id與用戶id組合成遠端位置
linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (secretFileContentJson["self_user_id"], linkRichMenuId)
print(linkMenuEndpoint)

# 設定消息基本安全憑證
linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息
lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
print(lineLinkMenuResponse)
print(lineLinkMenuResponse.text)

https://api.line.me/v2/bot/user/U8633692756652c60054e28e65760bf8b/richmenu/richmenu-63bc7a36bfe8fecc78e7915aa7307fa3
<Response [200]>
{}


In [7]:
'''

檢視用戶目前所綁定的菜單
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

#  https://api.line.me/v2/bot/user/{userId}/richmenu

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)


https://api.line.me/v2/bot/user/U8633692756652c60054e28e65760bf8b/richmenu
<Response [404]>
{"message":"the user has no richmenu","details":[]}


In [8]:
'''

解除選單與特定用戶的綁定
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)

https://api.line.me/v2/bot/user/U8633692756652c60054e28e65760bf8b/richmenu
<Response [200]>
{}


In [9]:
'''

檢視帳號內，有哪些選單
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 設定Line的遠端位置
listMenuEndpoint="https://api.line.me/v2/bot/richmenu/list"

# 設定消息的基本安全憑證
listMenuHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}    

# 發送消息告知
lineLisrMenuResponse = requests.get(listMenuEndpoint,headers=listMenuHeader)
print(lineLisrMenuResponse.text)

{"richmenus":[{"richMenuId":"richmenu-800a7802ab4496cfa908d5f13fbc1293","name":"區塊鏈自定義菜單","size":{"width":2500,"height":1686},"chatBarText":"查看更多資訊","selected":true,"areas":[{"bounds":{"x":5,"y":0,"width":824,"height":850},"action":{"type":"message","text":"*營地推薦"}},{"bounds":{"x":0,"y":850,"width":825,"height":818},"action":{"type":"message","text":"*食譜推薦"}},{"bounds":{"x":829,"y":5,"width":871,"height":849},"action":{"type":"message","text":"*拍照辨識"}},{"bounds":{"x":825,"y":854,"width":875,"height":814},"action":{"type":"message","text":"[::text:]多組織多位公證人"}},{"bounds":{"x":1700,"y":0,"width":800,"height":858},"action":{"type":"message","text":"*營地搜尋"}},{"bounds":{"x":1700,"y":858,"width":800,"height":810},"action":{"type":"message","text":"[::text:]more"}}]},{"richMenuId":"richmenu-c9c209337d1ced2b201a746cb01eb682","name":"區塊鏈自定義菜單","size":{"width":2500,"height":1686},"chatBarText":"查看更多資訊","selected":true,"areas":[{"bounds":{"x":5,"y":0,"width":824,"height":850},"action":{"type":"mes